# Create CMAQ-Ready File from Shapefile

---
    author: Barron H. Henderson
    date: 2020-04-25
    updated: 2020-04-29
---

This Notebook uses geopandas and PseudoNetCDF to create IOAPI-like files for CMAQ. Geopandas supports STRtree optimized searches and projection conversions. This simplifies the process to basic steps:

1. Define grid cells as GeoPandas.GeoDataframe
2. Process shapefile
    * Read in native projection.
    * Clip shapefile to grid
    * Optionally, custom extra processing
3. Perform grid cell intersections with shapefile polygons.
4. Output
    * Aggregate results to grid cell level
    * Find largest area contirbutor
    * Store results as variables
    * Tidy metadata


# Define Inputs

* Each input is required
* Update according to the documentation.
* Most users will not make edits anywhere else.

In [1]:
# shppath : str
#     Path to a shapefile or zip file containing a shapefile
#     wget http://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip
#     Use 10m for better resolution
shppath = 'ne_110m_admin_0_countries.zip'

# attrkey : str
#     Attribute of shapes in shapefile that defines groups of shapes. For
#     exmaple, ADM0_A3 defines countries in NaturalEarth's countries file.
attrkey = 'ADM0_A3'

# gdpath : str
#     Path to a GRIDDESC file
gdpath = 'EXAMPLE_GRIDDESC'

# gdnam : str
#     Name of grid definition within gdpath (e.g., 12US1, 108NHEMI2)
gdnam = '108US1'

# outdir : str
#     Directory to make outputs with file names like shppath.gdnam.IOAPI.nc
outdir = '.'

# outformat : str
#     Use eitehr NETCDF4_CLASSIC or NETCDF3_CLASSIC. 
#    'NETCDF3_CLASSIC' will make bigger files, but more compatible with older IOAPI
outformat = 'NETCDF4_CLASSIC'

# debug : bool
#     Plot intermediate and final results for understanding what happened.
debug = False # if true, plot result

# overwrite : bool
#     Default False, old results will not be overwritten
#     If True, rewrite results
overwrite = True


In [2]:
%matplotlib inline
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, box
import pyproj
import pycno
import PseudoNetCDF as pnc

warnings.simplefilter('ignore')

import geopandas as gpd

In [3]:
rootname = os.path.splitext(os.path.basename(shppath))[0]
outpath = f'{outdir}/{rootname}.{gdnam}.IOAPI.nc'
os.environ['IOAPI_ISPH'] = '6370000.0'

In [4]:
FILEDESC = f"""title: {outpath}
author: shapefile2cmaq
description: {attrkey} fractional area coverage and dominnant (DOM_{attrkey})
inputs:
 - Shapefile: {shppath}
 - Attribute: {attrkey}
 - GRIDDESC: {gdpath}
 - GDNAM: {gdnam}
tool_version: 1.0
file_version: 1.0
"""
print(FILEDESC)

title: ./ne_110m_admin_0_countries.108US1.IOAPI.nc
author: shapefile2cmaq
description: ADM0_A3 fractional area coverage and dominnant (DOM_ADM0_A3)
inputs:
 - Shapefile: ne_110m_admin_0_countries.zip
 - Attribute: ADM0_A3
 - GRIDDESC: EXAMPLE_GRIDDESC
 - GDNAM: 108US1
tool_version: 1.0
file_version: 1.0



In [5]:
if os.path.exists(outpath):
    if overwrite:
        os.remove(outpath)
    else:
        raise IOError(f'{outpath} already exists')


In [6]:
if gdpath == "EXAMPLE_GRIDDESC" and not os.path.exists(gdpath):
    with open(gdpath, 'w') as gdf:
        gdf.write("""' '
'LamCon_40N_97W'
 2         33.000        45.000       -97.000       -97.000        40.000
' '
'108US1'
'LamCon_40N_97W'  -2952000.000  -2772000.000    108000.000    108000.000  60   50   1
' '""")

In [7]:
gf = pnc.pncopen(gdpath, format='griddesc', GDNAM=gdnam, SDATE=1970001)
proj = gf.getproj()

## Define the grid cells

In [8]:
%%time
gridcells = []

I, J = np.meshgrid(np.arange(gf.NCOLS), np.arange(gf.NROWS))
hdx = gf.XCELL / 2
hdy = gf.YCELL / 2
COL = I * gf.XCELL + hdx
ROW = J * gf.YCELL + hdy

for (ri, ci), c in np.ndenumerate(COL):
    r = ROW[ri, ci]
    gridcells.append(box(c - hdx, r - hdy, c + hdx, r + hdx))


CPU times: user 37.8 ms, sys: 0 ns, total: 37.8 ms
Wall time: 41.4 ms


In [9]:
gdf = gpd.GeoDataFrame(dict(
    I=I.ravel(), J=J.ravel()
), geometry=gridcells, crs=proj.srs)

## Process shapefile

### Read in native projection

In [10]:
cdf = gpd.read_file(shppath)

In [11]:
# Optionally, derive an attribute. For example, here I transform UTC time zone offsets
# to create a variable whose values have no symbols
# cdf[attrkey] = cdf['utc_format'].astype(str).str.replace('±', '+').str.replace('UTC', '')
# cdf[attrkey] = cdf[attrkey].str.replace('+', 'POS_').str.replace('-', 'NEG_').str.replace(':', '')
# cdf[attrkey].unique()

In [12]:
if attrkey not in cdf.columns:
    raise KeyError(f'{attrkey} is not found in {shppath}; try {cdf.columns}')

### Clip to grid cells

In [13]:
if gdf.crs.to_cf()['grid_mapping_name'] == 'polar_stereographic':
    # polar stereographic polygons have problems in WGS84, but their centroids 
    # do not use centroids to create a point then buffer the points to create a
    # polygon then clipping to spherical space
    #
    # Heuristic assumption that the buffer needs to be in degrees
    # and the midlatitude degrees/m is 100km. Tripling for safety.
    buff = max(gf.XCELL, gf.YCELL) / 100e3 * 3
    centroids = gdf.geometry.centroid.to_crs(cdf.crs)
    swx, swy, nex, ney = centroids.total_bounds
    sedge = swy - buff
    # Block warning about buffering in lat/lon space. Since this is
    # very much intended, I am  hiding the warning
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        blobs = centroids.buffer(buff)

    mask = gpd.clip(
        blobs,
        gpd.GeoSeries(box(-180, sedge, 180, 90), crs=4326).to_crs(cdf.crs)
    ).unary_union
else:
    tmpdf = gdf.to_crs(cdf.crs)
    mask = tmpdf[tmpdf.geometry.is_valid].geometry.unary_union.buffer(1)

# Subset the spatial domain of the shapefile
vcdf = gpd.clip(cdf, mask).to_crs(gdf.crs)

In [14]:
if debug:
    gpd.GeoSeries(mask).plot()
    vcdf.plot()

### Extra processing?

* Here you could add "extra processing"
* For example, surfzone can be created from coastlines.
  * Coastline geometries would be lines instead of polygons.
  * To get the surfzone polygon, you would add a 50m buffer.
  * That would be a symmetric buffer, but only the ocean side is surfzone.
  * So, you would then clip that to the ocean.
* The result would be a polygon.

In [15]:
# ocnpath = '/work/ROMO/gis_rasters/NaturalEarth/downloads/ne_50m_ocean.zip'
# oceandf = gpd.clip(gpd.read_file(ocnpath), mask).to_crs(gdf.crs)
# vcdf = gpd.clip(vcdf.buffer(50), oceandf.buffer(0))
# vcdf

In [16]:
# For ne_50m_ocean.zip, consider spliting into pieces
# vcdf = vcdf.explode().reset_index(drop=True)

## Find intersection between grid cells and polygons

In [17]:
%%time
intx = gpd.sjoin(gdf, vcdf)


CPU times: user 63.9 ms, sys: 2.76 ms, total: 66.7 ms
Wall time: 66.7 ms


In [18]:
%%time
intx['right_geometry'] = vcdf.geometry.loc[intx.index_right].values
if not intx['right_geometry'].is_valid.all():
    intx['right_geometry'] = intx['right_geometry'].buffer(0)

intx['intersection_geometry'] = intx['geometry'].intersection(
    intx['right_geometry'], align=False
)
intx['intersection_area'] = (
    intx['intersection_geometry'].area / gf.XCELL / gf.YCELL
)

CPU times: user 208 ms, sys: 20.5 ms, total: 228 ms
Wall time: 324 ms


## Output

### Aggregate results to the GRID CELL level

In [19]:
attr_area = intx[
    [attrkey, 'I', 'J', 'intersection_area']
].groupby([attrkey, 'I', 'J', ], as_index=False).sum()
# Limit fraction area to 1.
attrkeys = list(attr_area[attrkey].unique())
outkeys = sorted(attrkeys)
attr_range = tuple(attr_area["intersection_area"].describe()[["min", "max"]])
print(f'Coverage By A {attrkey}: {attr_range}')


Coverage By A ADM0_A3: (3.6761891184579765e-06, 1.0)


In [20]:
totkey = f'TOT_{attrkey}'
total_area = attr_area.groupby(['I', 'J'], as_index=False).sum()
total_range = tuple(total_area["intersection_area"].describe()[["min", "max"]])
print(f'Coverage By All {attrkey}: {total_range}')


Coverage By All ADM0_A3: (1.223901484760318e-05, 1.0000000000000002)


### Find the largest area contributor (aka dominant)

In [21]:
domkey = f'DOM_{attrkey}'
dom_area = attr_area.sort_values(
    by='intersection_area', ascending=False
).groupby(['I', 'J'], as_index=False).head(1)
dom_area[domkey] = dom_area[attrkey].map(attrkeys.index)


### Store results as variables

In [22]:
for key in outkeys:
    areas = attr_area.query(f'{attrkey} == "{key}"')
    print(key, end='.')
    outvar = gf.copyVariable(gf.variables['DUMMY'], key=key, dtype='f')
    outvar.setncatts(dict(
        long_name=key.ljust(16), var_desc=key.ljust(80), units='1'.ljust(16)
    ))
    outvar[:] = 0.0
    outvar[0, 0, areas.J.values, areas.I.values] = areas.intersection_area.values
print('\nDone')

BHS.BLZ.CAN.COL.CUB.DOM.GTM.HND.HTI.JAM.MEX.NIC.PRI.USA.VEN.
Done


In [23]:
if len(outkeys) > 1:
    outvar = gf.copyVariable(gf.variables['DUMMY'], key=totkey, dtype='f')
    outvar.setncatts(dict(
        long_name=totkey.ljust(16), var_desc=totkey.ljust(80),
        units='1'.ljust(16)
    ))
    outvar[:] = 0
    outvar.var_desc = f'Total {attrkey} coverage'
    outvar[0, 0, total_area.J.values, total_area.I.values] = (
        total_area['intersection_area'].values
    )
    
    outvar = gf.copyVariable(gf.variables['DUMMY'], key=domkey, dtype='i')
    outvar.setncatts(dict(
        long_name=domkey.ljust(16), var_desc=domkey.ljust(80),
        units='1'.ljust(16)
    ))
    outvar[:] = -9999
    outvar[0, 0, dom_area.J.values, dom_area.I.values] = (
        dom_area[domkey].values
    )
    outvar.var_desc = f'Dominant {attrkey} ID see description'
    outvar.description = f'{dict(enumerate(attrkeys))}'
    outkeys.append(totkey)
    outkeys.append(domkey)
    FILEDESC = FILEDESC + f'{domkey}: {outvar.description}'

### Tidy Metadata

In [24]:
gf.SDATE = 1970001
outf = gf.subset(outkeys)
outf.FILEDESC = FILEDESC.ljust(80*60)[:80*60]

In [25]:
renamers = {key: key[:15] for key in outf.variables if len(key) >= 16}
if len(renamers) > 0:
    outf = outf.renameVariables(**renamers)
    outkeys = [renamers.get(key, key) for key in outkeys]
# Optionally, rename variables for IOAPI compliance
# outf = outf.renameVariables(
#     'REALLLLLLLLY_TOO_LONG_NAME': 'LONG_NAME'
# )

In [26]:
if debug:
    plt.colorbar(plt.pcolormesh(
        np.ma.masked_less(outf.variables[outkeys[-1]][0, 0], -1000)
    ))
    pycno.cno(proj=outf.getproj(withgrid=True)).draw()

In [27]:
outf.variables['TFLAG'][:] = 0
outf.save(outpath, complevel=1, format=outformat, verbose=0).close()